In [141]:
library(tidyverse)
library(reshape2)

In [142]:
library("RSQLite")

## connect to db
con <- dbConnect(drv=RSQLite::SQLite(), dbname="../input/soccer/database.sqlite")

## list all tables
tables <- dbListTables(con)

## exclude sqlite_sequence (contains table information)
tables <- tables[tables != "sqlite_sequence"]

lDataFrames <- vector("list", length=length(tables))

## create a data.frame for each table
for (i in seq(along=tables)) {
  lDataFrames[[i]] <- dbGetQuery(conn=con, statement=paste("SELECT * FROM '", tables[[i]], "'", sep=""))
}



In [143]:
match <- data.frame(lDataFrames[3])
team <- data.frame(lDataFrames[6])
# head(match,2)
# head(team,2)


In [144]:
match <- match %>% select(match_api_id,date,home_team_api_id,away_team_api_id,home_team_goal, away_team_goal)
team <- team %>% select(team_api_id,team_long_name)


In [145]:
match$home_score <- (match$home_team_goal - match$away_team_goal)
match$away_score <- - match$home_score
match$home_win_rate <- as.integer(as.logical(match$home_score>0))
match$away_win_rate <- as.integer(as.logical(match$away_score>0))
head(match,6)


,match_api_id,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_score,away_score,home_win_rate,away_win_rate
,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,492473,2008-08-17 00:00:00,9987,9993,1,1,0,0,0,0
2,492474,2008-08-16 00:00:00,10000,9994,0,0,0,0,0,0
3,492475,2008-08-16 00:00:00,9984,8635,0,3,-3,3,0,1
4,492476,2008-08-17 00:00:00,9991,9998,5,0,5,-5,1,0
5,492477,2008-08-16 00:00:00,7947,9985,1,3,-2,2,0,1
6,492478,2008-09-24 00:00:00,8203,8342,1,1,0,0,0,0


In [146]:
match_home <- match %>% select(match_api_id,date,home_team_api_id,home_score,home_win_rate) %>% 
  rename(
    team_api_id = home_team_api_id,
    score = home_score,
    win_rate = home_win_rate
    )
match_away <- match %>% select(match_api_id,date,away_team_api_id,away_score, away_win_rate) %>% 
  rename(
    team_api_id = away_team_api_id,
    score = away_score,
    win_rate = away_win_rate
    )

In [147]:
total <- rbind(match_home, match_away)
total$date <- as.Date(total$date)
head(arrange(total,match_api_id))

,match_api_id,date,team_api_id,score,win_rate
,<int>,<date>,<int>,<int>,<int>
1,483129,2008-08-09,8583,1,1
2,483129,2008-08-09,9830,-1,0
3,483130,2008-08-09,9827,1,1
4,483130,2008-08-09,7819,-1,0
5,483131,2008-08-09,9746,1,1
6,483131,2008-08-09,9831,-1,0


In [148]:
match14 = with(total, total[(date >= "2014-01-01" & date < "2015-01-01"),])
match15 = with(total, total[(date >= "2015-01-01" & date < "2016-01-01"),])
                            

In [149]:
df14 = subset(match14, select=-c(date))
df15 = subset(match15, select=-c(date))

In [150]:
team_ranking14_byscore <- aggregate(df14$score, by=list(team_api_id=df14$team_api_id), FUN=mean) %>% rename(score=x)
team_ranking15_byscore <- aggregate(df15$score, by=list(team_api_id=df15$team_api_id), FUN=mean) %>% rename(score=x)

In [151]:
team_ranking14_bywin <- aggregate(df14$win_rate, by=list(team_api_id=df14$team_api_id), FUN=mean) %>% rename(win_rate=x)
team_ranking15_bywin <- aggregate(df15$win_rate, by=list(team_api_id=df15$team_api_id), FUN=mean) %>% rename(win_rate=x)

In [156]:
team_ranking14 <- merge(team,team_ranking14_byscore, by="team_api_id",all.team_ranking14_byscore = TRUE)
team_ranking15 <- merge(team,team_ranking15_byscore,by="team_api_id",all.team_ranking14_byscore = TRUE)

team_ranking14 <- arrange(merge(team_ranking14,team_ranking14_bywin, by="team_api_id"),-win_rate)
team_ranking15 <- arrange(merge(team_ranking15,team_ranking15_bywin,by="team_api_id"),-win_rate)




ERROR: Error: arrange() failed at implicit mutate() step. 
[31m✖[39m Could not create a temporary column for `..1`.
[34mℹ[39m `..1` is `-win_rate`.


In [155]:
head(team_ranking14)
head(team_ranking15)

,team_api_id,team_long_name,score,win_rate
,<int>,<chr>,<dbl>,<dbl>
1,8633,Real Madrid CF,2.222222,0.7500000
2,9823,FC Bayern Munich,2.114286,0.8285714
3,9925,Celtic,1.868421,0.7368421
4,8634,FC Barcelona,1.729730,0.6486486
5,9772,SL Benfica,1.633333,0.8333333
6,8342,Club Brugge KV,1.571429,0.5714286


,team_api_id,team_long_name,score,win_rate
,<int>,<chr>,<dbl>,<dbl>
1,8634,FC Barcelona,2.105263,0.7894737
2,9772,SL Benfica,2.058824,0.7352941
3,9925,Celtic,1.948718,0.7692308
4,9823,FC Bayern Munich,1.852941,0.7647059
5,9773,FC Porto,1.764706,0.7941176
6,9847,Paris Saint-Germain,1.763158,0.7894737
